In [3]:
import pandas as pd
import numpy as np

import yfinance as yf
import finnhub

import time
from datetime import datetime as dt, timedelta
from dateutil.relativedelta import relativedelta

import json
import requests

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm

from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

# Read in the Tickers

In [8]:
stocks = pd.read_excel('Data/universe.xlsx')['symbol']

# Fetching the Data

### Stock Prices

In [9]:
# Convert pd Series of strings into one long string (that the format yf wants)
string_format_stocks = stocks.str.cat(sep=' ')

# Fetch the price data
prices = yf.download(string_format_stocks, start='2021-01-01', end=dt.today().strftime('%Y-%m-%d'))

# Extract only the adj close data
adj_close = prices['Adj Close']

[*********************100%***********************]  110 of 110 completed


In [10]:
### TEMPORARYLY USE ONLY GOOG DATA ###

GOOG_price = adj_close['GOOG'].to_frame()
GOOG_price.reset_index(inplace=True)
GOOG_price['Date'] = GOOG_price['Date'].astype(str)
GOOG_price.head()

,Date,GOOG
0,2021-01-01,NaN
1,2021-01-02,NaN
2,2021-01-03,NaN
3,2021-01-04,86.412003
4,2021-01-05,87.045998


### Industry Breakdown

In [ ]:
# Initialize finhub client
finnhub_client = finnhub.Client(api_key="ccn3d6iad3i1nkrepma0ccn3d6iad3i1nkrepmag")

In [721]:
# Fetch the company data from finhub
industries = []

for count, ticker in enumerate(stocks):

    try:
        tickerdata = finnhub_client.company_profile2(symbol=ticker)
        industries.append(tickerdata['finnhubIndustry'])
    except Exception:
        industries.append(np.nan)

In [785]:
# Count the number of occurences of each industry
industries = np.array(industries)
unique, counts = np.unique(industries, return_counts=True)

# Calculate the precentage of industries
percentages = np.round(counts/np.sum(counts) * 100, 2)
print('Breakdown by Indsutry:')
print()
industries_dict = dict(zip(unique, percentages))
display(pd.DataFrame.from_dict(industries_dict, orient='index', columns=['Percentage of Total']))

Breakdown by Indsutry:



,Percentage of Total
Banking,0.91
Beverages,0.91
Biotechnology,0.91
Building,1.82
Chemicals,0.91
Commercial Services & Supplies,1.82
Consumer products,0.91
Distributors,0.91
Electrical Equipment,0.91
Energy,1.82


### Read in Previous Week Data

In [767]:
prev_general_market_news = pd.read_csv('Data/general_market_news.csv', index_col=0)
prev_financial_news = pd.read_csv('Data/financial_news.csv', index_col=0)
prev_ticker_news = pd.read_csv('Data/ticker_news.csv', index_col=0)

In [771]:
# Get the latest date from each file to start fetching new news articles from that date
general_market_news_latest_date = pd.to_datetime(general_market_news['Date']).max().date()
financial_news_latest_date = pd.to_datetime(financial_news['Date']).max().date()
ticker_news_latest_date = pd.to_datetime(prev_ticker_news['Date']).max().date()

# Convert datetime.date to datetime.datetime
general_market_news_latest_date = dt.combine(general_market_news_latest_date, dt.min.time())
financial_news_latest_date = dt.combine(financial_news_latest_date, dt.min.time())
ticker_news_latest_date = dt.combine(ticker_news_latest_date, dt.min.time())

# Fetch the data until this day
END_DATE_general = general_market_news_latest_date
END_DATE_financial = financial_news_latest_date
END_DATE_ticker = ticker_news_latest_date

### Market News

In [740]:
market_news = finnhub_client.general_news('general', min_id=0)

In [793]:
# Parse the news articles
parsed_market_news = []

for article in market_news:
    headline = article['headline']
    summary = article['summary']
    date = dt.fromtimestamp(article['datetime'])
    
    parsed_market_news.append([date, headline, summary])

In [917]:
# Convert to DataFrame
columns = ['Date', 'Headline', 'Summary']
general_market_news = pd.DataFrame(parsed_market_news, columns=columns)

## Alpha Vantage

In [458]:
ALPHA_VANTAGE_KEY = '4GIL4K9ZN1NWE26G'

In [465]:
# # Federal Funds Interest Rate

# url = 'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=daily&apikey=' + ALPHA_VANTAGE_KEY
# r = requests.get(url)
# data = r.json()

# # Maybe will use later

In [701]:
# Create a list of dates with a weekly frequency
dates_timestamps = pd.date_range(start=END_DATE_financial, end=dt.today(), freq='W').tolist()
dates = list(map(dt.date, dates_timestamps))

In [732]:
# Helper function that extracts needed info from json object more efficiently
def extract_info_vantage(article):
    time_published = dt.strptime(article['time_published'][:8], '%Y%m%d')
    sentiment_score = article['overall_sentiment_score']
    sentiment_label = article['overall_sentiment_label']
    return [time_published, sentiment_score, sentiment_label]

# This is the response from the API when no articles are found
empty_response = {'Information': 'No articles found. Please adjust the time range or refer to the API documentation https://www.alphavantage.co/documentation#newsapi and try again.'}

In [775]:
# Fetch the following news

financial_market_news = []
economy_fiscal_news = []
economy_monetary_news = []
economy_macro_news = []

time_from = dates[0].strftime('%Y%m%d') + 'T0000' # start with the firsst date in the list

for date in dates[1:]:
    
    time_to = date.strftime('%Y%m%d') + 'T0000' # update the time_to

    # financial market news

    topic = 'financial_markets'

    url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&topics=' + topic \
            + '&time_from=' + time_from + '&time_to=' + time_to + '&limit=200&apikey=' + ALPHA_VANTAGE_KEY 
    result = requests.get(url)
    data = result.json()

    if data != empty_response:
        financial_market_news.extend(list(map(extract_info_vantage, data['feed']))) # add data to the list

    # Economy - Fiscal Policy news economy_fiscal

    topic = 'economy_fiscal'

    url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&topics=' + topic \
            + '&time_from=' + time_from + '&time_to=' + time_to + '&limit=200&apikey=' + ALPHA_VANTAGE_KEY 
    result = requests.get(url)
    data = result.json()

    if data != empty_response:
        economy_fiscal_news.extend(list(map(extract_info_vantage, data['feed']))) # add data to the list

    # Economy - Monetary Policy

    topic = 'economy_monetary'

    url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&topics=' + topic \
            + '&time_from=' + time_from + '&time_to=' + time_to + '&limit=200&apikey=' + ALPHA_VANTAGE_KEY 
    result = requests.get(url)
    data = result.json()

    if data != empty_response:
        economy_monetary_news.extend(list(map(extract_info_vantage, data['feed']))) # add data to the list

    # Economy - Macro/Overall

    topic = 'economy_macro'

    url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&topics=' + topic \
            + '&time_from=' + time_from + '&time_to=' + time_to + '&limit=200&apikey=' + ALPHA_VANTAGE_KEY 
    result = requests.get(url)
    data = result.json()

    if data != empty_response:
        economy_macro_news.extend(list(map(extract_info_vantage, data['feed']))) # add data to the list
    
    # now the time_from is the previous time to, so looking at the following week articles
    time_from = time_to 
    
    # unpaid subscription allows 5 calls per minute
    time.sleep(60)

In [777]:
# Convert lists of data to DataFrame objects
columns = ['Date', 'Sentiment Score', 'Sentiment Label']

financial_market_news_df = pd.DataFrame(financial_market_news, columns=columns)
financial_market_news_df['Topic'] = 'financial_market_news'                       # specify the topic of the df

economy_fiscal_news_df = pd.DataFrame(economy_fiscal_news, columns=columns)
economy_fiscal_news_df['Topic'] = 'economy_fiscal_news'                           # specify the topic of the df

economy_monetary_news_df = pd.DataFrame(economy_monetary_news, columns=columns)
economy_monetary_news_df['Topic'] = 'economy_monetary_news'                       # specify the topic of the df

economy_macro_news_df = pd.DataFrame(economy_macro_news, columns=columns)
economy_macro_news_df['Topic'] = 'economy_macro_news'                             # specify the topic of the df

# Concatenate these dfs into one long df
financial_news = pd.concat([financial_market_news_df, economy_fiscal_news_df, economy_monetary_news_df, economy_macro_news_df])

# Reset index (otherwise each index is repeated 4 times)
financial_news.reset_index(drop=True, inplace=True)

## Polygon.io

In [ ]:
POLYGON_KEY = 'Chz4dhUuzmumD5YcQeSpI7M_JgItlGJc'

In [728]:
# Helper function that gets the dates of fetched news articles
def get_dates(dic):
    return dic['published_utc'][:10]

# Helper function that extracts needed info from json object more efficiently
def extract_info_polygon(article):
    time_published = article['published_utc'][:10]
    title = article['title']
    try:
        description = article['description']
    except:
        description = np.nan        
    return [time_published, title, description]

In [744]:
# Fetch news per ticker
request_counter = 0
news = {}

for num, ticker in enumerate(stocks):
    
    print(ticker, num)
    
    TICKER = ticker
    DATE = dt.today().strftime('%Y-%m-%d')
    
    news[TICKER] = []
    
    while dt.strptime(DATE, '%Y-%m-%d') > END_DATE_ticker:
        
        print('running for date:', DATE)
                        
        # Unpaid subscription allows 5 calls per minute
        if request_counter == 5:
            time.sleep(60)
            request_counter = 0
        
        # Fetch the news articles
        result = requests.get('https://api.polygon.io/v2/reference/news?order=desc&ticker=' + TICKER + 
                              '&published_utc.lte=' + DATE + '&limit=1000&apiKey=' + POLYGON_KEY)
        all_articles = json.loads(result.text)['results']
        
        request_counter += 1
        
        # Append needed info from the json object to a list
        news[ticker].extend(list(map(extract_info_polygon, all_articles)))
            
        # Get the latest date in the dictionary of articles
        dates = np.array(list(map(get_dates, all_articles)))
        try:
            new_date = np.unique(dates)[0]
        except IndexError:
            new_date = END_DATE.strftime('%Y-%m-%d')# if there is no more historical data for this stock, just set the 
                                                    # new_date to END_DATE so that the loop goes on to the next stock
        
        # If new_date == DATE, manually adjust the new_date to be the previous date        
        if new_date == DATE:
            new_date = (dt.strptime(new_date, '%Y-%m-%d') - timedelta(days=1)).strftime('%Y-%m-%d')
                
        DATE = new_date


In [754]:
# Parse the articles into df format
parsed_news = []

for ticker in news.keys():
    for article in news[ticker]:
        date, title, summary = article
        parsed_news.append([ticker, date, title, summary])
        
columns = ['Ticker', 'Date', 'Title', 'Summary']        
news_df = pd.DataFrame(parsed_news, columns=columns)
news_df['Date'] = pd.to_datetime(news_df['Date'])

In [918]:
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

# Ticker news
scores = news_df['Summary'].apply(analyzer.polarity_scores).tolist()
df_scores = pd.DataFrame(scores)
news_df = news_df.join(df_scores)

# General market news
scores = general_market_news['Summary'].apply(analyzer.polarity_scores).tolist()
df_scores = pd.DataFrame(scores)
general_market_news = general_market_news.join(df_scores)

In [756]:
# Concat the new articles with the previous ones
ticker_news = pd.concat([prev_news, news_df])
general_market_news = pd.concat([prev_general_market_news, general_market_news])
financial_news = pd.concat([prev_financial_news, financial_news])

# Drop duplicates (some stocks might not have many articles and so fetching new articles might also bring some already existing ones)
general_market_news.drop_duplicates(subset=['Date', 'Headline', 'Summary'], inplace=True)
financial_news.drop_duplicates(subset=['Date', 'Sentiment Score', 'Sentiment Label', 'Topic'], inplace=True)
ticker_news.drop_duplicates(subset=['Ticker', 'Date', 'Title', 'Summary'], inplace=True)

# Sort articles by Ticker, Date
general_market_news.sort_values(by=['Date'], ascending=False, inplace=True)
financial_news.sort_values(by=['Topic', 'Date'], ascending=False, inplace=True)
ticker_news.sort_values(by=['Ticker', 'Date'], ascending=False, inplace=True)

# Reset index
general_market_news.reset_index(drop=True, inplace=True)
financial_news.reset_index(drop=True, inplace=True)
ticker_news.reset_index(drop=True, inplace=True)

### DataFrames:

<li> general_market_news </li>
<li> financial_news </li>
<li> ticker_news </li>

In [919]:
display(general_market_news.head())
display(general_market_news.shape)
display(financial_news.head())
display(financial_news.shape)
display(ticker_news.head())
display(ticker_news.shape)

,Date,Headline,Summary,neg,neu,pos,compound
0,2022-10-03 12:14:31,3 takeaways from our daily meeting: Stocks jum...,"The Investing Club holds its ""Morning Meeting""...",0.000,1.000,0.000,0.0000
1,2022-10-03 11:54:08,This classic investment strategy is on track f...,The 60/40 portfolio is supposed to give invest...,0.000,0.833,0.167,0.6369
2,2022-10-03 11:52:20,Photos show massive recovery days after Hurric...,Insurers could face costs of up to $57 billion...,0.209,0.791,0.000,-0.7003
3,2022-10-03 11:22:20,Satellite operator Viasat up 35% after selling...,Satellite operator Viasat is selling a piece o...,0.000,0.841,0.159,0.3400
4,2022-10-03 11:14:45,We're moving money from an oil name to a beaut...,This trade will result in a nearly dollar for ...,0.000,0.825,0.175,0.6249


(100, 7)

,Date,Sentiment Score,Sentiment Label,Topic
0,2022-03-04,0.176178,Somewhat-Bullish,financial_market_news
1,2022-03-04,0.288176,Somewhat-Bullish,financial_market_news
2,2022-03-04,-0.058874,Neutral,financial_market_news
3,2022-03-04,0.025208,Neutral,financial_market_news
4,2022-03-05,-0.032453,Neutral,financial_market_news


(24226, 4)

,Ticker,Date,Title,Summary,neg,neu,pos,compound
0,XOM,2022-10-02,5 Top Stocks for October,This basket of growth and dividend stocks has ...,0.000,0.686,0.314,0.6705
1,XOM,2022-10-01,Facebook parent Meta is no longer one of 10 mo...,Meta Platforms Inc. is furthering its descent ...,0.049,0.951,0.000,-0.1531
2,XOM,2022-09-30,"Bearish On Exxon, Chevron? This Inverse ETF Tr...",Direxion Energy Bear 2X Shares (NYSE: ERY) was...,0.000,0.896,0.104,0.9100
3,XOM,2022-09-30,The Zacks Analyst Blog Highlights Berkshire Ha...,"Berkshire Hathaway, Exxon Mobil, Walmart, Alib...",0.000,0.909,0.091,0.2023
4,XOM,2022-09-30,Cracker Barrel Old Country Store and Mohawk In...,Cracker Barrel Old Country Store and Mohawk In...,0.000,1.000,0.000,0.0000


(39654, 8)

### Write the DataFrames to csv

In [779]:
general_market_news.to_csv('Data/general_market_news.csv')
financial_news.to_csv('Data/financial_news.csv')
ticker_news.to_csv('Data/ticker_news.csv')

# Data Preprocessing

In [977]:
# Convert Date columns to strings
general_market_news['Date'] = general_market_news['Date'].dt.date.astype(str)
financial_news['Date'] = financial_news['Date'].dt.date.astype(str)
ticker_news['Date'] = ticker_news['Date'].dt.date.astype(str)

In [975]:
# Group Sentiment Score per day
general_market_news_grouped = general_market_news.groupby(by=['Date'])['compound'].mean().to_frame()
financial_news_score = financial_news.groupby(by=['Topic', 'Date'])['Sentiment Score'].mean().to_frame()
financial_news_label = financial_news.groupby(by=['Topic', 'Date'])['Sentiment Label'].agg(pd.Series.mode).to_frame()
financial_news_grouped = financial_news_score.merge(financial_news_label, on=['Topic', 'Date'], how='left')
ticker_news_grouped = ticker_news.groupby(by=['Ticker', 'Date'])['compound'].mean().to_frame()

# Reset indeces
general_market_news_grouped.reset_index(inplace=True)
financial_news_grouped.reset_index(inplace=True)
ticker_news_grouped.reset_index(inplace=True)

In [1003]:
# Create a list of dates with a daily frequency from 2021-01-01 to today
date_range = pd.date_range(dt(2021,1,1), dt.today(), freq='D').to_list()
date_range = list(map(dt.date, date_range))
data_index = list(map(str, date_range))

# Create an empty DataFrame which will contain all our feature data
data = pd.DataFrame({'Date': data_index})

In [1004]:
# General Market News
general_market_news_grouped.columns = ['Date', 'general_market_news_compound']
data = data.merge(general_market_news_grouped, on=['Date'], how='left')

In [1005]:
# Financial News
unique_topics = financial_news_grouped['Topic'].unique()

for topic in unique_topics:
    
    # Get a df for one topic at a time
    topic_news = financial_news_grouped.loc[financial_news_grouped['Topic'] == topic]
    
    # Rename the columns
    topic_news = topic_news.rename(columns={'Sentiment Score': f'{topic}_sentiment_score', 'Sentiment Label': f'{topic}_sentiment_label'})
    topic_news.drop(columns=['Topic'], inplace=True)

    # Join with the data DataFrame
    data = data.merge(topic_news, on=['Date'], how='left')

In [1006]:
# Ticker News (ONLY FOR GOOGLE)
GOOG_data = ticker_news_grouped.loc[ticker_news_grouped['Ticker'] == 'GOOG']
GOOG_data = GOOG_data.drop(columns=['Ticker'])
GOOG_data.columns = ['Date', 'ticker_news_compound']
data = data.merge(GOOG_data, on=['Date'], how='left')

In [4]:
### TEMPORARY ###
data = pd.read_csv('data.csv', index_col=0)
data.tail()

,Date,general_market_news_compound,economy_fiscal_news_sentiment_score,economy_fiscal_news_sentiment_label,economy_macro_news_sentiment_score,economy_macro_news_sentiment_label,economy_monetary_news_sentiment_score,economy_monetary_news_sentiment_label,financial_market_news_sentiment_score,financial_market_news_sentiment_label,ticker_news_compound
636,2022-09-29,NaN,0.024190,Neutral,0.065411,Neutral,-0.179354,Somewhat-Bearish,0.135896,Somewhat-Bullish,-0.170700
637,2022-09-30,NaN,0.012914,Neutral,0.033396,Neutral,-0.006839,Neutral,0.135896,Somewhat-Bullish,0.258917
638,2022-10-01,-0.240186,0.089276,Somewhat-Bullish,-0.015609,Neutral,0.011346,Neutral,0.132295,Somewhat-Bullish,0.090900
639,2022-10-02,-0.055226,0.089276,Somewhat-Bullish,-0.015609,Neutral,0.011346,Neutral,0.132295,Somewhat-Bullish,0.113600
640,2022-10-03,-0.025882,0.089276,Somewhat-Bullish,-0.015609,Neutral,0.011346,Neutral,0.132295,Somewhat-Bullish,0.113600


In [11]:
# Forward fill missing values
data = data.ffill(axis=0)

# Add quarters
data['quarter'] = pd.PeriodIndex(data['Date'], freq='Q')

# Add price data
data = data.merge(GOOG_price, on=['Date'], how='left')

# Keep only trading days
data = data.loc[data['GOOG'].notna()]

# Convert price to return
data['GOOG'] = data['GOOG'].pct_change()

# Drop general_market_news_compound column b/c it's almost all NaN
data.drop(columns=['general_market_news_compound'], inplace=True)

# Drop NaNs (drops many columns)
data.dropna(inplace=True)

In [12]:
# Encode categorical variables
encoder = LabelEncoder()
data['economy_fiscal_news_sentiment_label'] = encoder.fit_transform(data['economy_fiscal_news_sentiment_label'])
data['economy_macro_news_sentiment_label'] = encoder.fit_transform(data['economy_macro_news_sentiment_label'])
data['economy_monetary_news_sentiment_label'] = encoder.fit_transform(data['economy_monetary_news_sentiment_label'])
data['financial_market_news_sentiment_label'] = encoder.fit_transform(data['financial_market_news_sentiment_label'])

In [13]:
data.tail()

,Date,economy_fiscal_news_sentiment_score,economy_fiscal_news_sentiment_label,economy_macro_news_sentiment_score,economy_macro_news_sentiment_label,economy_monetary_news_sentiment_score,economy_monetary_news_sentiment_label,financial_market_news_sentiment_score,financial_market_news_sentiment_label,ticker_news_compound,quarter,GOOG
634,2022-09-27,0.086522,0,0.040090,1,-0.179354,1,0.135896,2,0.027780,2022Q3,-0.007287
635,2022-09-28,0.072263,0,0.054712,1,-0.179354,1,0.135896,2,0.099890,2022Q3,0.027016
636,2022-09-29,0.024190,0,0.065411,1,-0.179354,1,0.135896,2,-0.170700,2022Q3,-0.026305
637,2022-09-30,0.012914,0,0.033396,1,-0.006839,0,0.135896,2,0.258917,2022Q3,-0.019778
640,2022-10-03,0.089276,2,-0.015609,1,0.011346,0,0.132295,2,0.113600,2022Q4,0.032761


In [14]:
# Divide dataset into X and y
features = ['economy_fiscal_news_sentiment_score',
       'economy_fiscal_news_sentiment_label',
       'economy_macro_news_sentiment_score',
       'economy_macro_news_sentiment_label',
       'economy_monetary_news_sentiment_score',
       'economy_monetary_news_sentiment_label',
       'financial_market_news_sentiment_score',
       'financial_market_news_sentiment_label', 'ticker_news_compound']

y = data['GOOG']
X = data[features]

# Scale the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [15]:
# Divide X and y into train and test sets
y_train = y[:120]
X_train = X[:120]

y_test = y[120:]
X_test = X[120:]

# Finding the Best Model

In [ ]:

params = {'n_estimators':[10,50,100,250], 'max_depth':[5,10,20], 'class_weight':[None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]}

clf = RandomForestClassifier(random_state=42)
gs = GridSearchCV(clf, params, scoring='roc_auc', n_jobs=-1)
gs.fit(X_train, y_train)

print("Best set of Parameters",gs.best_params_)
print("Best Score",gs.best_score_)

In [ ]:
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'LogisticRegression': LogisticRegression(),
    'SVR': SVR(),
    'RandomForestRegressor': RandomForestRegressor(),
    'ExtraTreesRegressor': ExtraTreesRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'MLPRegressor': MLPRegressor()
}

In [92]:
np.linspace(0.1, 10, 20)

array([ 0.1       ,  0.62105263,  1.14210526,  1.66315789,  2.18421053,
        2.70526316,  3.22631579,  3.74736842,  4.26842105,  4.78947368,
        5.31052632,  5.83157895,  6.35263158,  6.87368421,  7.39473684,
        7.91578947,  8.43684211,  8.95789474,  9.47894737, 10.        ])

In [93]:
params = {'alpha': np.linspace(0.1, 10, 20)}

In [16]:
model = Ridge(alpha=0)
reg = model.fit(X_train, y_train)
display(reg.score(X_train, y_train))

y_pred = reg.predict(X_test)

display(mean_squared_error(y_test, y_pred))

0.04334282307425952

0.0004586633263684654

In [17]:
params = {'alpha': np.linspace(0.01, 10, 200)}
model = Ridge()

grid_search = GridSearchCV(model, param_grid=params, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': array([ 0.01      ,  0.06020101,  0.11040201,  0.16060302,  0.21080402,
        0.26100503,  0.31120603,  0.36140704,  0.41160804,  0.46180905,
        0.51201005,  0.56221106,  0.61241206,  0.66261307,  0.71281407,
        0.76301508,  0.81321608,  0.86341709,  0.91361809,  0.9638191 ,
        1.0140201 ,  1.06422111,  1.11442211,  1.16462312,  1.21482412,
        1.26502513,  1.31...
        8.29316583,  8.34336683,  8.39356784,  8.44376884,  8.49396985,
        8.54417085,  8.59437186,  8.64457286,  8.69477387,  8.74497487,
        8.79517588,  8.84537688,  8.89557789,  8.94577889,  8.9959799 ,
        9.0461809 ,  9.09638191,  9.14658291,  9.19678392,  9.24698492,
        9.29718593,  9.34738693,  9.39758794,  9.44778894,  9.49798995,
        9.54819095,  9.59839196,  9.64859296,  9.69879397,  9.74899497,
        9.79919598,  9.84939698,  9.89959799,  9.94979899, 10.        ])},
             scoring='r2')

In [18]:
grid_search.best_score_

-0.18257697473978424

In [19]:
grid_search.best_params_

{'alpha': 10.0}